In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 12.0 MB/s eta 0:00:00


In [2]:
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

df.fillna(df.mean(), inplace=True)

print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-05-15 15:27:26,848] A new study created in memory with name: no-name-733f91a3-bb8e-4219-984b-dac2d13f7f50
[I 2025-05-15 15:27:29,121] Trial 0 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 150, 'max_depth': 4}. Best is trial 0 with value: 0.7597765363128491.
[I 2025-05-15 15:27:32,340] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 137, 'max_depth': 12}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-05-15 15:27:34,406] Trial 2 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 109, 'max_depth': 14}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-05-15 15:27:37,637] Trial 3 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 195, 'max_depth': 15}. Best is trial 3 with value: 0.7765363128491621.
[I 2025-05-15 15:27:39,922] Trial 4 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 83, 'max_depth': 5}. Best is trial 3 with value: 0.7765363

In [7]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 124, 'max_depth': 18}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


### Sampler In OPTUNA

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-05-15 15:51:45,175] A new study created in memory with name: no-name-b94ebe94-787b-4ddf-8b00-639f54d959f2
[I 2025-05-15 15:51:46,738] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 161, 'max_depth': 12}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-05-15 15:51:47,497] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 59, 'max_depth': 18}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-05-15 15:51:48,645] Trial 2 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 130, 'max_depth': 7}. Best is trial 2 with value: 0.7802607076350093.
[I 2025-05-15 15:51:49,063] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 56, 'max_depth': 17}. Best is trial 2 with value: 0.7802607076350093.
[I 2025-05-15 15:51:51,158] Trial 4 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 137, 'max_depth': 16}. Best is trial 2 with value: 0.7802607

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 130, 'max_depth': 7}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [13]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [14]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [15]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-05-15 16:02:36,678] A new study created in memory with name: no-name-9b6daf9d-68e5-4f0b-a4bc-72e246d96d40
[I 2025-05-15 16:02:37,512] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-05-15 16:02:38,931] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-05-15 16:02:39,424] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-05-15 16:02:39,965] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-05-15 16:02:40,518] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [16]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [17]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


### OPTUNA Visualization

In [18]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [19]:
# 1. Optimization History
plot_optimization_history(study).show()

In [20]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [21]:
# 3. Slice Plot
plot_slice(study).show()

In [22]:
# 4. Contour Plot
plot_contour(study).show()

In [23]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

### Optimizing Multiple ML Models

In [26]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [27]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [28]:

# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-05-15 16:20:56,034] A new study created in memory with name: no-name-75527c9e-173b-4c1c-9f5b-88d5de7d5bb1
[I 2025-05-15 16:20:56,077] Trial 0 finished with value: 0.7094972067039106 and parameters: {'classifier': 'SVM', 'C': 9.733571533108723, 'kernel': 'poly', 'gamma': 'scale'}. Best is trial 0 with value: 0.7094972067039106.
[I 2025-05-15 16:20:56,104] Trial 1 finished with value: 0.7839851024208566 and parameters: {'classifier': 'SVM', 'C': 0.4590497860128306, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 1 with value: 0.7839851024208566.
[I 2025-05-15 16:21:00,044] Trial 2 finished with value: 0.7504655493482307 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 168, 'learning_rate': 0.13312418752005, 'max_depth': 14, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.7839851024208566.
[I 2025-05-15 16:21:00,853] Trial 3 finished with value: 0.7765363128491619 and parameters: {'classifier': 'RandomForest', 'n_estimators': 187

In [29]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11357448681754279, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [30]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.709497,2025-05-15 16:20:56.036411,2025-05-15 16:20:56.077434,0 days 00:00:00.041023,9.733572,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.783985,2025-05-15 16:20:56.078622,2025-05-15 16:20:56.104699,0 days 00:00:00.026077,0.459050,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.750466,2025-05-15 16:20:56.105803,2025-05-15 16:21:00.043893,0 days 00:00:03.938090,NaN,NaN,GradientBoosting,NaN,NaN,0.133124,14.0,3.0,8.0,168.0,COMPLETE
3,3,0.776536,2025-05-15 16:21:00.045017,2025-05-15 16:21:00.853448,0 days 00:00:00.808431,NaN,False,RandomForest,NaN,NaN,NaN,6.0,2.0,2.0,187.0,COMPLETE
4,4,0.731844,2025-05-15 16:21:00.854696,2025-05-15 16:21:06.615093,0 days 00:00:05.760397,NaN,NaN,GradientBoosting,NaN,NaN,0.068077,15.0,3.0,9.0,292.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.787709,2025-05-15 16:21:35.771687,2025-05-15 16:21:35.802614,0 days 00:00:00.030927,0.183047,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.778399,2025-05-15 16:21:35.803641,2025-05-15 16:21:36.654314,0 days 00:00:00.850673,NaN,False,RandomForest,NaN,NaN,NaN,5.0,7.0,5.0,202.0,COMPLETE
97,97,0.789572,2025-05-15 16:21:36.655513,2025-05-15 16:21:36.684991,0 days 00:00:00.029478,0.147864,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.787709,2025-05-15 16:21:36.686020,2025-05-15 16:21:36.717608,0 days 00:00:00.031588,0.102026,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [31]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
GradientBoosting,11
RandomForest,10


In [32]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.747926
RandomForest,0.765363
SVM,0.774085


In [33]:
# 1. Optimization History
plot_optimization_history(study).show()

In [34]:
# 3. Slice Plot
plot_slice(study).show()

In [35]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [39]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-05-15 16:25:56,018] A new study created in memory with name: no-name-7a15f34b-3418-4607-8d39-75fffa422c15


[0]	train-mlogloss:1.00011	eval-mlogloss:0.99330
[1]	train-mlogloss:0.77722	eval-mlogloss:0.75725
[2]	train-mlogloss:0.61819	eval-mlogloss:0.58954
[3]	train-mlogloss:0.53114	eval-mlogloss:0.49117
[4]	train-mlogloss:0.48471	eval-mlogloss:0.44252
[5]	train-mlogloss:0.42066	eval-mlogloss:0.37246
[6]	train-mlogloss:0.37445	eval-mlogloss:0.32308
[7]	train-mlogloss:0.37323	eval-mlogloss:0.32368
[8]	train-mlogloss:0.33393	eval-mlogloss:0.27971
[9]	train-mlogloss:0.31855	eval-mlogloss:0.25979
[10]	train-mlogloss:0.30563	eval-mlogloss:0.24641
[11]	train-mlogloss:0.30532	eval-mlogloss:0.24724
[12]	train-mlogloss:0.30339	eval-mlogloss:0.24548
[13]	train-mlogloss:0.30294	eval-mlogloss:0.24505
[14]	train-mlogloss:0.30244	eval-mlogloss:0.24345
[15]	train-mlogloss:0.30214	eval-mlogloss:0.24261
[16]	train-mlogloss:0.30108	eval-mlogloss:0.23870
[17]	train-mlogloss:0.30030	eval-mlogloss:0.23564
[18]	train-mlogloss:0.29996	eval-mlogloss:0.23647
[19]	train-mlogloss:0.29964	eval-mlogloss:0.23653
[20]	train

[I 2025-05-15 16:26:04,112] Trial 0 finished with value: 1.0 and parameters: {'lambda': 2.9227831540491894e-08, 'alpha': 0.01030202876276488, 'eta': 0.2189678725541205, 'gamma': 0.001558996911773764, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.6900845111530045, 'colsample_bytree': 0.49652205100904734}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:1.01224	eval-mlogloss:1.02676
[1]	train-mlogloss:0.85353	eval-mlogloss:0.86033
[2]	train-mlogloss:0.72709	eval-mlogloss:0.72109
[3]	train-mlogloss:0.64864	eval-mlogloss:0.63728
[4]	train-mlogloss:0.57551	eval-mlogloss:0.56584
[5]	train-mlogloss:0.49892	eval-mlogloss:0.48277
[6]	train-mlogloss:0.45308	eval-mlogloss:0.44053
[7]	train-mlogloss:0.42021	eval-mlogloss:0.40798
[8]	train-mlogloss:0.37188	eval-mlogloss:0.35287
[9]	train-mlogloss:0.34142	eval-mlogloss:0.32203
[10]	train-mlogloss:0.30754	eval-mlogloss:0.28386
[11]	train-mlogloss:0.28044	eval-mlogloss:0.25124
[12]	train-mlogloss:0.27122	eval-mlogloss:0.24573
[13]	train-mlogloss:0.24517	eval-mlogloss:0.21498
[14]	train-mlogloss:0.23876	eval-mlogloss:0.21042
[15]	train-mlogloss:0.22616	eval-mlogloss:0.19655
[16]	train-mlogloss:0.21397	eval-mlogloss:0.18524
[17]	train-mlogloss:0.20171	eval-mlogloss:0.17148
[18]	train-mlogloss:0.19312	eval-mlogloss:0.16352
[19]	train-mlogloss:0.18486	eval-mlogloss:0.15496
[20]	train

[I 2025-05-15 16:26:06,273] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.04794039569682033, 'alpha': 1.0694382203173115e-07, 'eta': 0.14597207869264991, 'gamma': 4.6101708288472903e-08, 'max_depth': 8, 'min_child_weight': 2, 'subsample': 0.5595577384293356, 'colsample_bytree': 0.4145763627376159}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.98722	eval-mlogloss:0.99060
[1]	train-mlogloss:0.83134	eval-mlogloss:0.81429


[I 2025-05-15 16:26:16,808] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:0.94792	eval-mlogloss:0.94404
[1]	train-mlogloss:0.82241	eval-mlogloss:0.79744
[2]	train-mlogloss:0.72386	eval-mlogloss:0.69802


[I 2025-05-15 16:26:16,912] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 2.9227831540491894e-08, 'alpha': 0.01030202876276488, 'eta': 0.2189678725541205, 'gamma': 0.001558996911773764, 'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.6900845111530045, 'colsample_bytree': 0.49652205100904734}
Best accuracy: 1.0


In [37]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 2.3 MB/s eta 0:00:00


In [40]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()